# Lesson 01: Getting topographic data

*This lesson made by Simon M Mudd and last updated 20/11/2021*

In this lesson we will grab some topographic data from the internet using [Opentopography](https://opentopography.org/).

[Opentopography](https://opentopography.org/) hosts a large number of topographic datasets and is supported by the National Science Foundation of the United States. 

*LESSON SHORTCUT*

If you want to run a full analysis of a place of your choosing just modify the (tbc)

## What you can find on OpenTopography

You can see what is on OpenTopograhy by [following this link](https://portal.opentopography.org/dataCatalog)

But briefly it has:
    
* Several global topographic datasets. 
    * Shuttle Radar Topography Mission (SRTM) data, at approxamately 30 and 90 m grid resolutions. Based on radar data. 
    * The NASADEM, which is a reprocessed version of the SRTM data (so theoretically better quality than SRTM from the same base data).
    * The ALOS World 3D 30 m dataset. This is based on optical data made using photogrammetry from the ALOS satellites. It is a downscaled version of the 5 m dataset (the difference is that the 30 m one is free).
    * The Copernicus DEM, which is 30m grid comes from a variety of sources but much of it from the TanDEM-X radar satellites.
* Vast quantities of high resolution data from lidar (cm scale point clouds and 1 m grid resolution DEMs).
    * These data are mostly from the United States but the site also has selected data from around the world (for example there are a number of datasets from New Zealand)

OpenTopography has an api for its global dataset so it is, by some wide margin, the most convinent place to source global data at 30 m grid spacing or coarser. 

Here is the link to the [api key](https://portal.opentopography.org/apidocs/#/Public/getGlobalDem)

However, we will use a tool to grab this data called `lsdviztools` which was written at the University of Edinburgh.

## First import some stuff we need

We need to update a package called `lsdviztools` that has a subcomponent for grabbing opentopography data. 

In [1]:
#!pip install lsdviztools --upgrade

Double check that the version is correct. For this lesson we need version 0.4.6 or above.

In [1]:
import lsdviztools
lsdviztools.__version__

'0.4.7'

Now import a bunch of other stuff for the lesson

In [2]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw
import pandas as pd
import geopandas as gpd
import cartopy as cp
import cartopy.crs as ccrs
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

## Grab some data

Okay, lets grab some data! We use a tool in `lsdviztools.lsdbasemaptools` called the `ot_scraper` (`ot` is for opentopography.org).

Below is some code for grabbing data from opentopography. It uses the syntax:

`DEM_name = bmt.ot_scraper(parameters=...)`

where you need to give it some paramaters. These are things like what sort of data you want (most people will use the SRTM 30 metre data) and you also tell it the lower left and the upper right corners, using latitude and longitude. You can get these from goole earth by right clicking on the map and selecting "what's here". 

In this example I use the Rio Aguas in southern Spain (a place we run field courses). 

We designate the parameters by giving the lower left and upper right coordinates of the area from which you want the DEM. 

The format of these is so that you can right click on google maps and click on the coordinates, which will copy them to your clipboard, and you can paste them in to the lower left and upper right coordinates.

These coordinates are lists so you need them in square brackets. 

In [3]:
lower_left = [36.990554387425014, -2.318307057720176]
upper_right = [37.23367133834253, -1.8425313329873874]

Okay, below is the syntax for downloading the data.

In [5]:
Aguas_DEM = bmt.ot_scraper(source = "SRTMGL1",
                           lower_left_coordinates = lower_left, 
                           upper_right_coordinates = upper_right,
                           prefix = "rio_aguas") 
Aguas_DEM.print_parameters()
Aguas_DEM.download_pythonic()

I am taking your coordinates from the lower left list
I am taking your coordinates from the upper right list
The source is: SRTMGL1
The west longitude is: -2.318307057720176
The east longitude is: -1.8425313329873874
The south latitude is: 37.23367133834253
The north latitude is: 36.990554387425014
The path is: ./
The prefix is: rio_aguas
The resolution is: 30
I am going to download the following for you:
https://portal.opentopography.org/API/globaldem?demtype=SRTMGL1&south=36.990554387425014&north=37.23367133834253&west=-2.318307057720176&east=-1.8425313329873874&outputFormat=GTiff
This might take a little while, depending on the size of the file. 
The filename will be:
./rio_aguas_SRTMGL1.tif
The path and file without path are:
./  rio_aguas_SRTMGL1.tif
Finished downloading


('./rio_aguas_SRTMGL1.tif', './', 'rio_aguas_SRTMGL1.tif')

In [ ]:
Aguas_DEM = bmt.ot_scraper(source = "SRTM30",
                           longitude_W = -2.318307057720176, longitude_E = -1.8425313329873874, 
                           latitude_S = 36.990554387425014, latitude_N = 37.23367133834253,
                           prefix = "rio_aguas") 
Aguas_DEM.print_parameters()
Aguas_DEM.download_pythonic()

We need to convert this data into a format that can be understood by *lsdtopotools*, which is a topographic analysis package developed at the University of Edinburgh. 

In [ ]:
DataDirectory = "./"
RasterFile = "rio_aguas_SRTMGL1.tif"
gio.convert4lsdtt(DataDirectory, RasterFile,minimum_elevation=0.01,resolution=30)

# Now lets do some basic topographic analysis

For this to work you need *lsdtopotools* installed on your system. But if you are using either Edinburgh's noteable service or the *lsdtopotools* docker container, it is already installed. If you are not using one of these, you will need to install it using conda (see *lsdtopotools* installation instructions. 

We will extract some topogaphic metrics using `lsdtopotools`. 

The `lsdtt_parameters` are the various parameters that you can use to run an analysis. We will discuss these later. For now, we will just follow this recipe. 

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true",  
                    "surface_fitting_radius" : "60",
                    "print_slope" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "rio_aguas_SRTMGL1_UTM",
                                 write_prefix= "rio_aguas_SRTMGL1_UTM",
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

# Plot some data

We are now going to do some simple plots using a mapping package that we put together. There are more general ways to visualise data, but this makes pretty pictures quickly.  

In [ ]:
%matplotlib inline
Base_file = "rio_aguas_SRTMGL1_UTM"
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology")